In [1]:
import os, re, glob, json
import ollama
from pymilvus import MilvusClient
from pypdf import PdfReader

from typing import List

In [2]:
# ---------- Config ----------
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "phi3:latest")
EMBED_MODEL  = os.getenv("EMBED_MODEL",  "nomic-embed-text")
DB_PATH      = os.getenv("MILVUS_LITE_PATH", "./milvus.db")
COLLECTION   = os.getenv("MILVUS_COLLECTION", "docs")
CHUNK_CHARS  = int(os.getenv("CHUNK_CHARS", "1200"))
CHUNK_OVER   = int(os.getenv("CHUNK_OVERLAP", "200"))
TOP_K        = int(os.getenv("TOP_K", "5"))

In [27]:

def read_pdf(path:str):

    '''
    Read input PDF file, pagewise, and return text :str
    '''
    
    out = []
    r = PdfReader(path)
    
    for page in r.pages:

        t = page.extract_text() or ""
        out.append(t)

    return "\n".join(out)


def load_docs(folder: str):

    '''
    Load the .pdf and .txt documents
    '''
    docs = []

    for path in glob.glob(os.path.join(index,"**","*"),recursive=True):

        if not os.path.isfile(path):

            continue

        ext = os.path.splitext(path)[1].lower().strip()

        try:
            if ext in [".pdf"]:
    
                text = read_pdf(path)
    
            elif ext in [".txt"]:
    
                continue
    
            else:
                continue
    
            if text.strip():
    
                docs.append({"doc_id": os.path.basename(path),
                            "text": text
                            })
        except Exception as e:
            print(e)
            print(f"[skip] {path}")
            
            
    return docs

            

        
    
def chunk_text(text: str, size=1200, overlap=200):

    '''
    Chunk the input text for the minimum given size (default: 1200 chars), with overlap (default: 200 chars) for the next
    '''

    sents = re.split(r'(?<=[\.\!\?])\s+', text.strip())

    # print("sentence level split text loss chgeck")
    # print(len(text),[len(i) for i in sents], sum([len(i) for i in sents]) )

    chunks, cur = [], ""

    for s in sents:

        
        if len(s)+len(cur) <= size:

            cur = cur+(" " if cur else "") + s

        else:

            if cur:
                chunks.append(cur)

            tail = cur[-overlap:] if overlap >0 else ""

            cur = (tail + " " + s).strip()

    if cur:
        chunks.append(cur)

    return chunks



def embed_texts(texts: List[str]):
    '''
    Embed the input batch of chunks
    '''

    resp = ollama.embed(model=OLLAMA_MODEL, input=texts)
    return resp["embeddings"]



client = MilvusClient(DB_PATH)

def ensure_collection(dimension: int):

    collections_list = client.list_collections()
    print("collections_list : ", collections_list)
    
    if COLLECTION not in collections_list:

        
        client.create_collection(
            
            collection_name = COLLECTION,
            dimension = dimension,
            metric_type="COSINE",
            index_params = {"index_type": "AUTOINDEX"},
            auto_id=True
            
        )

        print(f"[milvus] created collection {COLLECTION} , dim={dimension}")
        
                

def index_folder(folder: str):

    '''

    '''
    
    docs = load_docs(folder)

    if not docs:

        print("[index] no documents found")

    print([i["doc_id"] for i in docs])


    #---------Chunk & Embed Documents -------------------------------

    batch  = []
    meta_batch = []
    sample_emb_dim = None
    records = []
    
    for d in docs:

        chunks = chunk_text(d["text"], CHUNK_CHARS, CHUNK_OVER)
        
        for i, ch in enumerate(chunks):

            batch.append(ch)

            meta_batch.append({"doc_id":d["doc_id"],
                              "chunk_id": i,
                               "text": ch
                              })
            
            if len(batch) >= 64:

                print("batch length touched: 64")
                vecs = embed_texts(batch)

                if sample_emb_dim is None:

                    sample_emb_dim = len(vecs[0])
                    ensure_collection(sample_emb_dim)

                for m,v in zip(meta_batch, vecs):

                    records.append({"vector":v, **m})
                
                batch, meta_batch = [], []
        
    if batch:
        print("Final batch length: ", len(batch))
        vecs = embed_texts(batch)

        print("Vectors Length",len(vecs))
        
        if not client.has_collection(COLLECTION):
            print("Collection Not found")
            ensure_collection(len(vecs[0]))

        for m,v in zip(meta_batch, vecs):
            records.append({"vector":v, **m})
        
        batch, meta_batch = [], []

    if records:

        client.insert(collection_name=COLLECTION, data=records)
        print(f"records are inserted into collection: {COLLECTION}, records length: {len(records)}")


In [28]:
def chat_once(question: str):

    '''
    
    '''

    SYSMTEM = ''' You are a helpful research assistant.
    - When answering user questions, call the 'retrive_from_mivlus' tool to gather relevant context when needed.
    - Cite each quote with {doc_id;chunk:id}.
    - If nothing is found, say so and ask the user to add documents.
    '''

    pass

    

    

In [29]:
# agent initialize

# if __name__ == "__main__":

#     import argparse

#     ap = argparse.ArgumentParser()

#     ap.add_argument("--index",type= str,help="Path to folder of docs to ingest")
#     ap.add_argument("--ask", type= str, help="ASk a question, to get an answer based on documents using LLM")

#     args = ap.parse_args()

#     if args.index:
#         index_folder(args.index)
#     if args.ask:
#         print(chat_once(args.ask))


index = "data"
ask = "Tell me about radhika"

if index:
    index_folder(index)
# if ask:
#     print(chat_once(ask))




['Ramesh_Naik Resume.pdf', 'Abhishek_Prasanna_Walavalkar Resume.pdf', 'Radhika_Ganesh Resume.pdf']
Final batch length:  17
Vectors Length 17
Collection Not found
collections_list :  []
[milvus] created collection docs , dim=3072
records are inserted into collection: docs, records length: 17


In [31]:
# client.list_collections()

NameError: name 'records' is not defined

In [25]:
# client.flush(COLLECTION)

In [26]:
# client.drop_collection(COLLECTION)